In [1]:
from jax_mlm_helpers import build_vocabulary
from jax_mlm_helpers import apply_random_masking
from jax_mlm_helpers import pad_and_crop_to_maximum_length
from ../utils_display import pc

SyntaxError: invalid syntax (1180341470.py, line 4)

In [ ]:
import spacy
from collections import Counter
nlp = spacy.load("en_core_web_sm")
from transformers import AutoTokenizer

In [ ]:
with open("../local_datasets/wikipedia_man_o_war.txt", "r") as fid:
    text = fid.read()

In [ ]:
text = text.replace("\n", " ")
while "  " in text:
    text = text.replace("  ", " ")

In [ ]:
sentences = [i for i in nlp(text).sents]
for i, s in enumerate(sentences):
    pc(i, s)

In [ ]:
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokens = []
for s in sentences:        
    tokens.append(tokenizer.tokenize(str(s)))
print(tokens[0])

In [ ]:
dico_word2index, dico_index2word = build_vocabulary(texts=tokens)

In [ ]:
words_frequency = {}
for sentence in sentences:
    for word in sentence:
        
        if str(word) in words_frequency:
            words_frequency[str(word)] += 1
        else:
            words_frequency[str(word)] = 1

words_frequency = sorted(words_frequency.items(), key=lambda x:x[1], reverse=True)

number_of_examples = 20
pc("{} most frequent words".format(number_of_examples), words_frequency[:number_of_examples])
pc("{} least frequent words".format(number_of_examples), words_frequency[-number_of_examples:])

In [ ]:
input_sentence = sentences[14]
word_tokens = tokenizer.tokenize(str(input_sentence))
input_indices = [dico_word2index.get(w, '[UNK]') for w in word_tokens]

pc("Input sentence", input_sentence)
pc("Word tokens", word_tokens)
pc("Input indices", input_indices)

In [ ]:
masking_probability = 0.15
label_for_unmasked_values = -100
maximum_sequence_length = 50
mask_index = dico_word2index["[MASK]"]
pad_index = dico_word2index["[PAD]"]

In [ ]:
input_indices, mask, masked_indices, labels = apply_random_masking(                                                                                              
    input_indices=input_indices,                                                                                                 
    index_for_masked_values=mask_index,
    label_for_unmasked_values=label_for_unmasked_values,                                                                                 
     masking_probability=masking_probability)

print("Before padding / cropping")
pc("Input indices", input_indices)
pc("Mask", mask)
pc("Masked indices", masked_indices)
pc("Labels", labels)

In [ ]:
input_indices = pad_and_crop_to_maximum_length(input_indices, padding_value=pad_index, maximum_sequence_length=maximum_sequence_length)
mask = pad_and_crop_to_maximum_length(mask, padding_value=1, maximum_sequence_length=maximum_sequence_length)
masked_indices = pad_and_crop_to_maximum_length(masked_indices, padding_value=pad_index, maximum_sequence_length=maximum_sequence_length)
labels = pad_and_crop_to_maximum_length(labels, padding_value=-100, maximum_sequence_length=maximum_sequence_length)

print("After padding / cropping")
pc("Input indices", input_indices)
pc("Mask", mask)
pc("Masked indices", masked_indices)
pc("Labels", labels)